# Code playground
This notebook is a small sandbox where half baked ideas can live. Code snippets that were developed but might be useful saterr can go here.

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchtext
import tqdm
from torchnlp.metrics import get_moses_multi_bleu
from torchtext.data import Field, BucketIterator
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

import tensorflow as tf
import tensorflow_datasets as tfds
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO

import linecache
import sys
import os
import re
import random
import time
# import streamtologger
# streamtologger.redirect(target="./logs-playground.log")

In [2]:
from datetime import datetime
def super_print(filename):
    '''filename is the file where output will be written'''
    def wrap(func):
        '''func is the function you are "overriding", i.e. wrapping'''
        def wrapped_func(*args,**kwargs):
            '''*args and **kwargs are the arguments supplied 
            to the overridden function'''
            #use with statement to open, write to, and close the file safely
            with open(filename,'a') as outputfile:
                now = datetime.now()
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                outputfile.write("[{}] ".format(dt_string))
                outputfile.write(" ".join(str(x) for x in args))
                outputfile.write("\n")
            #now original function executed with its arguments as normal
            return func(*args,**kwargs)
        return wrapped_func
    return wrap

print = super_print('logs-playground.txt')(print)

In [20]:
torch.tensor([1,4,3,6,2]).argsort()

tensor([0, 4, 2, 1, 3])

In [21]:
rand_transformer_model = nn.Transformer().to("cuda") # uses default hyperparameters
src = torch.rand((10, 32, 512)).to("cuda") # [src_seq_length, batch_size, embedding_size]
tgt = torch.rand((20, 32, 512)).to("cuda") # [tgt_seq_length, batch_size, embedding_size]
rand_transformer_model(src, tgt).shape # [tgt_seq_length, batch_size, embedding_size]

torch.Size([20, 32, 512])

In [22]:
rand_transformer_model.decoder.layers[-1]

TransformerDecoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): Linear(in_features=512, out_features=512, bias=True)
  )
  (multihead_attn): MultiheadAttention(
    (out_proj): Linear(in_features=512, out_features=512, bias=True)
  )
  (linear1): Linear(in_features=512, out_features=2048, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=2048, out_features=512, bias=True)
  (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (norm3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
  (dropout3): Dropout(p=0.1, inplace=False)
)

In [31]:
q = torch.rand((20, 32, 512))
k = torch.rand((10, 32, 512))
v = torch.rand((10, 32, 512))
att = nn.MultiheadAttention(512, 2)
attn_output = att(q, k, v)
# attn_output

## Self-made beam search code
This code runs on paralelised batches and beams, it's fast, but doesn't have the appropriate stopping conditions.

In [ ]:
def beam_search_decode_batch_ids(encoder_input, beam_size=3, max_seq_length=50):
    batch_len = encoder_input.shape[1]
    sos_id = TGT_TEXT.vocab.stoi["<sos>"]
    decoder_input = torch.zeros((1, beam_size * batch_len), dtype=torch.long, device=device).fill_(sos_id)
    log_probs = torch.zeros((beam_size * batch_len,1))
    print("log_probs:", log_probs)
    vocab_size = len(TGT_TEXT.vocab.itos)
    
    encoder_input = encoder_input.view(-1,1).repeat(1,beam_size).view(-1,beam_size * batch_len) # this tiles the input to the beam size * batch size
    print("tiled input:", encoder_input)
    
    for i in range(max_seq_length):
        output = model(encoder_input, decoder_input)
        print(output.shape)
        last_pred = output[-1].softmax(1)
        print("last_pred shape:", last_pred.shape)
        log_predictions = last_pred.log()
        seq_log_probs = log_predictions + log_probs.repeat(1,vocab_size)
        print("seq_log_probs:", seq_log_probs)
        
        seq_log_probs_positions = seq_log_probs.view(batch_len,-1).argsort(1)[:,-beam_size:]
        print("seq_log_probs_positions:", seq_log_probs_positions)
        next_ids = seq_log_probs_positions.reshape(1, batch_len*beam_size) % vocab_size
        print("next_ids:", next_ids)
        log_probs = seq_log_probs.view(batch_len,-1).gather(1,seq_log_probs_positions).view(beam_size * batch_len,1)
        print("log_probs:",log_probs)
        
        for batch_idx in range(seq_log_probs_positions.shape[0]):
            for seq_choice_idx in range(seq_log_probs_positions.shape[1]):
                seq_choice = seq_log_probs_positions[batch_idx,seq_choice_idx] // vocab_size
                decoder_input[:,batch_idx*batch_len + seq_choice_idx] = decoder_input[:,batch_idx*batch_len + seq_choice]
        
        decoder_input = torch.cat((decoder_input, next_ids))
    return decoder_input
        
#         last_pred = output[-1].argsort(dim=1)[:,:beam_size]

sent1 = ["<sos>"] + SRC_TEXT.preprocess("create array") + ["<eos>"] + ["<pad>"]
sent2 = ["<sos>"] + SRC_TEXT.preprocess("if exists then") + ["<eos>"]
src_ids = SRC_TEXT.numericalize([sent1, sent2], device=device)
print("input ids:", src_ids)
beam_search_decode_batch_ids(src_ids, max_seq_length=5)

## Non batch single node beam search
This is the original beam search from online simply adapted to suit the transformerr architecture.

In [ ]:
class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length
        
    def __lt__(self, other):
        return True

    def eval(self, alpha=1.0):
        reward = 0
        beta = 4.0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward

In [ ]:
def beam_decode(model, encoder_states):
    '''
    :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
    :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
    :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
    :return: decoded_batch
    '''

    beam_width = 10
    topk = 3  # how many sentence do you want to generate
    decoded_batch = []
    
    SOS_token = TGT_TEXT.vocab.stoi["<sos>"]
    EOS_token = TGT_TEXT.vocab.stoi["<eos>"]
    MAX_LENGTH = 7
    
    batch_size = encoder_states.shape[1]

    # decoding goes sentence by sentence
    for idx in range(batch_size):
        encoder_input = encoder_states[:, idx].view(-1,1)
        
        # Start with the start of the sentence token
        decoder_input = torch.LongTensor([[SOS_token]]).to(device)

        # Number of sentence to generate
        endnodes = []
        number_required = min((topk + 1), topk - len(endnodes))

        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_input, None, SOS_token, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1

        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 400: break

            # fetch the best node
            score, n = nodes.get()
#             decoder_input = n.wordid
            decoder_input = n.h

            if n.wordid == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue

            # decode for one step using decoder
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)
#             print(encoder_input)
#             print(decoder_input)
            decoder_output = model(encoder_input, decoder_input)
            last_token_logits = decoder_output[-1]
            last_token_logs = last_token_logits.log_softmax(1)
            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(last_token_logs, beam_width)
            nextnodes = []

            for new_k in range(beam_width):
                decoded_t = indexes[0][new_k]
                log_p = log_prob[0][new_k].item()
                decoder_input = torch.cat((decoder_input,decoded_t.view(1,-1)))
                node = BeamSearchNode(decoder_input, n, decoded_t.cpu().item(), n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increase qsize
            qsize += len(nextnodes) - 1

        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]

        utterances = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance = []
            utterance.append(n.wordid)
            # back trace
            while n.prevNode != None:
                n = n.prevNode
                utterance.append(n.wordid)

            utterance = utterance[::-1]
            utterances.append(utterance)

        decoded_batch.append(utterances)

    return decoded_batch

sent1 = ["<sos>"] + SRC_TEXT.preprocess("call the options.get method with string 'CULL_FREQUENCY' and integer 3 as arguments, use the string 'cull_frequency' and previous result as the arguments for the call to the params.get method, substitute the result for cull_frequency.") + ["<eos>"] + ["<pad>"]
# sent2 = ["<sos>"] + SRC_TEXT.preprocess("if not,") + ["<eos>"]
src_ids = SRC_TEXT.numericalize([sent1], device=device)
# print("input ids:", src_ids)
outs = beam_decode(model, encoder_states=src_ids)

for b in outs:
    for sent in b:
        print([TGT_TEXT.vocab.itos[id] for id in sent])

## Adaptation of node based beam search to parallelise batches
This version comes from an already working version for a single batch. But there is a problem since when decodng different batches and passing them to the transformer, they all need to be the same length which doesn't work since there could be a node made in the past with a better probability score.
It doesn't look too obvious from what I see to speed it up while rretaining the theoretical benefits of being able to explore the full tree of options.

### The solution
You just need to add padding to the decoded sequences that are shorter and keep track of the position that you need to take the output token from. easy.

In [3]:
def beam_decode(model, encoder_states):
    '''
    :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
    :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
    :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
    :return: decoded_batch
    '''

    beam_width = 10
    topk = 3  # how many sentence do you want to generate
    decoded_batch = []
    
    batch_size = encoder_states.shape[1]
    
    SOS_token = TGT_TEXT.vocab.stoi["<sos>"]
    EOS_token = TGT_TEXT.vocab.stoi["<eos>"]
    MAX_LENGTH = 7

    # decoding goes all batches at the same time
    encoder_input = encoder_states

    # Start with the start of the sentence token
    decoder_input = torch.LongTensor([[SOS_token]]).to(device)

    # Number of sentence to generate
    batch_endnodes = [[] for i in range(batch_size)]
    number_required = topk

    # starting node -  hidden vector, previous node, word id, logp, length
    batch_node = [BeamSearchNode(decoder_input, None, SOS_token, 0, 1) for i in range(batch_size)]
    batch_nodes = [PriorityQueue() for i in range(batch_size)]

    # start the queue
    for nodes, node in zip(batch_nodes, batch_node):
        nodes.put((-node.eval(), node))
        
    batch_qsize = [1 for i in range(batch_size)]

    # start beam search
    while True:
        # give up when decoding takes too long for the first batch, placeholder for now
        print(batch_qsize)
        if batch_qsize[0] > 200: break

        # fetch the best node
        best_nodes = [nodes.get() for nodes in batch_nodes]
#         score, n = nodes.get()
#             decoder_input = n.wordid
#         decoder_input = n.h
        
        finished_nodes = [True if n.wordid == EOS_token and n.prevNode != None else False for (score, n) in best_nodes]
        print(finished_nodes)
        
        working_nodes = []
        working_node_id = 0
        working_nodes_idx = []
        for endnodes, (score, n) in zip(batch_endnodes, best_nodes):
            if n.wordid == EOS_token and n.prevNode != None and len(endnodes) < number_required:
                endnodes.append((score, n))
            else:
                working_nodes.append((score, n))
                working_nodes_idx.append(working_node_id)
            working_node_id += 1
        
        if all([len(endnodes) >= number_required for endnodes in batch_endnodes]):
            break
        
        num_working_nodes = len(working_nodes)
        print(working_nodes)
        
        step_encoder_input = encoder_input[:,:num_working_nodes].view(-1,num_working_nodes)
        print("[n.h for (score, n) in working_nodes] shape:", [n.h.shape for (score, n) in working_nodes])
        step_decoder_input = torch.cat([n.h for (score, n) in working_nodes],dim=1)
#         print("step_encoder_input shape:", step_encoder_input.shape)
        step_decoder_output = model(step_encoder_input, step_decoder_input)
        step_token_logits = step_decoder_output[-1]
        
        print(step_token_logits)

        # PUT HERE REAL BEAM SEARCH OF TOP
        log_prob, indexes = torch.topk(step_token_logits, beam_width)
        print(log_prob.shape)
        
    
        nextnodes = []
        
        for batch_id in working_nodes_idx:
            for new_k in range(beam_width):
                decoded_t = indexes[batch_id][new_k]
                log_p = log_prob[batch_id][new_k].item()
#                 print("decoder_input shape", step_decoder_input.shape)
#                 print("step_decoder_input shape:", step_decoder_input[:,batch_id].view(-1,1).shape)
#                 print("decoded_t shape:", decoded_t.view(1,-1).shape)
                decoder_input = torch.cat((step_decoder_input[:,batch_id].view(-1,1),decoded_t.view(1,-1)), dim=0)
#                 print("decoder_input shape:", decoder_input.shape)
                node = BeamSearchNode(decoder_input, working_nodes[batch_id], decoded_t.cpu().item(), n.logp + log_p, n.leng + 1)
                score = -node.eval()
                batch_nodes[batch_id].put((score, node))
                batch_qsize[batch_id] += 1
        
        # put them into queue
#         for i in range(len(nextnodes)):
#             score, nn = nextnodes[i]
#             nodes.put((score, nn))
#             # increase qsize
#         qsize += len(nextnodes) - 1

    # choose nbest paths, back trace them
    if len(endnodes) == 0:
        endnodes = [nodes.get() for _ in range(topk)]

    utterances = []
    for score, n in sorted(endnodes, key=operator.itemgetter(0)):
        utterance = []
        utterance.append(n.wordid)
        # back trace
        while n.prevNode != None:
            n = n.prevNode
            utterance.append(n.wordid)

        utterance = utterance[::-1]
        utterances.append(utterance)

    decoded_batch.append(utterances)

    return decoded_batch

sent1 = ["<sos>"] + SRC_TEXT.preprocess('try,') + ["<eos>"] + ["<pad>"]
sent2 = ["<sos>"] + SRC_TEXT.preprocess("if not,") + ["<eos>"]
src_ids = SRC_TEXT.numericalize([sent1, sent2], device=device)
# print("input ids:", src_ids)
[len(x) for x in beam_decode(model, encoder_states=src_ids)[0]]

NameError: name 'SRC_TEXT' is not defined

## Debug dataset

In [ ]:
# debug_data = [
#     ("my favourite foods are banana and toast","would you like banana and toast ?"),
#     ("my favourite foods are eggs and bacon and beans","would you like eggs and bacon and beans ?"),
#     ("my favourite food is chocolate","would you like chocolate ?"),
#     ("my favourite food is avocado","would you like avocado ?")
# ]

# other_data = [
#     ("what age is she ?", "she is 8 years old"),
#     ("what age is he ?", "he is 4 years old"),
#     ("how old are you ?", "i am 22 years old"),
#     ("how old am i ?", "you are 28 years old")
# ]

# SRC_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')
# TGT_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')

# train_dataset = val_dataset = samples_to_dataset(other_data, SRC_TEXT, TGT_TEXT)

# # train_dataset, val_dataset = dataset.split([0.7,0.3])

## Adding Noise to the decoding process during training

In [85]:
noise_e = 0.4
noise_mask = (torch.rand(10,10) > noise_e).float()

mask = (torch.triu(torch.ones(10,10))).transpose(0, 1)
mask = torch.mul(mask, noise_mask)
v = (torch.sum(mask, dim=-1) == 0).float()

fix_mask = torch.zeros(10,10)
fix_mask[:,0] = 1.0
v = v.repeat(10, 1).transpose(0,1)
fix_mask = torch.mul(fix_mask,v)

print(fix_mask)
mask += fix_mask


mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
mask

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., -inf, -inf, -inf, 0., -inf, -inf, -inf, -inf, -inf],
        [-inf, 0., 0., -inf, -inf, 0., -inf, -inf, -inf, -inf],
        [0., -inf, 0., 0., -inf, -inf, 0., -inf, -inf, -inf],
        [-inf, 0., 0., -inf, 0., 0., 0., 0., -inf, -inf],
        [-inf, 0., 0., 0., 0., -inf, 0., -inf, 0., -inf],
        [0., -inf, 0., 0., -inf, 0., -inf, 0., -inf, -inf]])

## Pytorch target masking transformer bug
There is a bug in the implementation of the transformer attention mask during decoding. It being produced the other way round, paying attention to parts of the sentence that appear at the end. It is a simple to fix at removing ==1

In [34]:
model = nn.Transformer()
model.generate_square_subsequent_mask(4)

tensor([[0., 0., 0., 0.],
        [-inf, 0., 0., 0.],
        [-inf, -inf, 0., 0.],
        [-inf, -inf, -inf, 0.]])

In [82]:
sz = 4
mask = (torch.triu(torch.ones(sz, sz))==1).transpose(0, 1).float()
print(mask)
mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
mask

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])


tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [74]:
a = (torch.ones(10)==1)
a.masked_fill(a == 1, float('-inf'))

tensor([True, True, True, True, True, True, True, True, True, True])

In [146]:
from queue import PriorityQueue
a = PriorityQueue()
a.put(torch.tensor(1))
a.put(torch.tensor(-2))
a.get()

tensor(-2)

In [128]:
a = torch.tensor([[1],[2],[3]])
b = torch.tensor([[3],[4]])
rough_input = [a,b]
m = max([t.shape[0] for t in rough_input])
print(m)
z = torch.zeros((m,2)).fill_(7)
for i in range(2):
    length = rough_input[i].shape[0]
    z[:length,i] = rough_input[i].view(-1)
print(z)

3
tensor([[1., 3.],
        [2., 4.],
        [3., 7.]])


In [140]:
outputs = torch.tensor([[[1],[5]],[[2],[6]],[[3],[7]],[[4],[8]]]).transpose(0,1)
p_mask = [2,3]

for l in outputs[]:
    print(l)

tensor([[1],
        [2],
        [3],
        [4]])
tensor([[5],
        [6],
        [7],
        [8]])


In [148]:
torch.tensor([1.,2,3]).log_softmax(0)

tensor([-2.4076, -1.4076, -0.4076])

# Python AST doodles

In [15]:
import ast
import astor

In [26]:
tree = ast.parse("print('foo','bar')")
ast.dump(tree)

"Module(body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Str(s='foo'), Str(s='bar')], keywords=[]))])"

In [18]:
astor.codetoast("print('foo')")

In [24]:
from copy_gen_transformer import Transformer
model = Transformer()